In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d pranavraikokte/covid19-image-dataset

 99% 156M/158M [00:01<00:00, 106MB/s]
100% 158M/158M [00:01<00:00, 83.2MB/s]


In [ ]:
from tensorflow import keras

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/covid19-image-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
train_data_dir = "/content/Covid19-dataset/train"
test_data_dir = "/content/Covid19-dataset/test"
img_width, img_height = 224, 224
batch_size = 32

In [ ]:
train_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


Found 251 images belonging to 3 classes.


In [ ]:
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


Found 66 images belonging to 3 classes.


In [ ]:
base_model = VGG16(weights='imagenet', include_top=False,input_shape=(img_width, img_height,3))

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=test_generator,
    validation_steps=len(test_generator),
    epochs=50
)

Epoch 1/50
8/8 [==============================] - 12s 718ms/step - loss: 18.5895 - accuracy: 0.1474 - val_loss: 1.0772 - val_accuracy: 0.3939
Epoch 2/50
8/8 [==============================] - 7s 908ms/step - loss: 2.0714 - accuracy: 0.3825 - val_loss: 1.4207 - val_accuracy: 0.3030
Epoch 3/50
8/8 [==============================] - 7s 807ms/step - loss: 1.7257 - accuracy: 0.2789 - val_loss: 1.2424 - val_accuracy: 0.3939
Epoch 4/50
8/8 [==============================] - 7s 838ms/step - loss: 1.2139 - accuracy: 0.1434 - val_loss: 1.1079 - val_accuracy: 0.3939
Epoch 5/50
8/8 [==============================] - 7s 829ms/step - loss: 1.2422 - accuracy: 0.3147 - val_loss: 1.0845 - val_accuracy: 0.3939
Epoch 6/50
8/8 [==============================] - 6s 793ms/step - loss: 1.1724 - accuracy: 0.4422 - val_loss: 1.0968 - val_accuracy: 0.3939
Epoch 7/50
8/8 [==============================] - 7s 866ms/step - loss: 1.1242 - accuracy: 0.4422 - val_loss: 1.0967 - val_accuracy: 0.3939
Epoch 8/50
8/8 [==

In [ ]:
model.evaluate(test_generator)

3/3 [==============================] - 1s 167ms/step - loss: 0.6597 - accuracy: 0.7879


[0.6596887707710266, 0.7878788113594055]

In [ ]:
base_model2 = VGG16(weights='imagenet', include_top=True, input_shape=(224, 224, 3))

In [ ]:
base_model2.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
from keras.models import Model
extraction_model=Model(inputs=base_model2.input,outputs= base_model2.get_layer('fc1').output)

In [ ]:
extraction_model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
train_features = extraction_model.predict(train_generator)

8/8 [==============================] - 5s 491ms/step


In [ ]:
test_features = extraction_model.predict(test_generator)

3/3 [==============================] - 1s 215ms/step


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [ ]:
clf = SVC(C=1.0, kernel='rbf')
clf.fit(train_features, train_generator.classes)

SVC()

In [ ]:
predicted_labels = clf.predict(test_features)
accuracy = np.mean(predicted_labels == test_generator.classes)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 84.85%
